Training phase 1 and 2

In [1]:
import tensorflow as tf
from tensorflow import keras as keras
import numpy as np
from numpy.random import rand
from numpy.random import randint
from numpy.random import randn
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Input
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import UpSampling2D
import keras
from keras import layers
import keras.backend as K
from keras.layers import LeakyReLU

In [2]:
# creates: band-alignment.png
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.analysis.structure_matcher import StructureMatcher
#from ase.db import connect
#import ase
atom_dict1=['na','H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg','Al','Si','P','S','Cl','Ar','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Kr','Rb','Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Te','I','Xe','Cs','Ba','La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg','Tl','Pb','Bi','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr','Rf','Db','Sg','Bh','Hs','Mt','Ds','Rg','Cn','Nh','Fl','Mc','Lv','Ts','Og']
#removing nobels 
atom_dict=['na','H','Li','Be','B','C','N','O','F','Na','Mg','Al','Si','P','S','Cl','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Rb','Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Te','I','Cs','Ba','La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg','Tl','Pb','Bi','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr','Rf','Db','Sg','Bh','Hs','Mt','Ds','Rg','Cn','Nh','Fl','Mc','Lv','Ts','Og']

In [12]:
## Load Real crystal images

In [7]:
x_real_train=np.load('2D_Real_Train.npy')
#np.shape(x_real_train)

Load Topo Data

In [15]:
x_train_topo=np.load('x_train_topo.npy')
y_train_topo=np.load('y_train_topo.npy')

In [12]:
def generate_topo_samples(xdataset, ydataset, n_samples):
    ix= randint(0, xdataset.shape[0], n_samples)
    X=xdataset[ix]
    y=ydataset[ix]
    y = np.expand_dims(y,1)
    #y = np.ones((n_samples,1))
    return X,y

In [14]:
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [15]:
def generate_real_samples(dataset, n_samples):
    ix= randint(0, dataset.shape[0], n_samples)
    X=dataset[ix]
    y = np.ones((n_samples,1))
    return X,y

In [16]:
def generate_fake_samples(n_samples):
    X=generate_latent_points(512*80,n_samples)
    X=X.reshape((n_samples,160,256))/(np.max(np.abs(X)))
    X=(X+1)/2
    y = np.zeros((n_samples,1))
    return X,y

In [17]:
def define_discriminator():
    input_img = keras.Input(shape=( 80, 512, 1))
    x = layers.Conv2D(4, 2, strides = (2,4),  padding='same')(input_img)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(32, 2,strides = (2,2),  padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(32, 3, strides = (2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Flatten()(x)
    output= layers.Dense(1, activation='sigmoid')(x)
    crstldisc = keras.Model(input_img, output)
    crstldisc.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #crstldisc.summary()
    return crstldisc

In [18]:
def define_topo_discriminator():
    input_img = keras.Input(shape=( 80, 512, 1))
    x = layers.Conv2D(4, 2, strides = (2,4),  padding='same')(input_img)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(32, 2,strides = (2,2),  padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(32, 3, strides = (2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Flatten()(x)
    output= layers.Dense(1, activation='sigmoid')(x)
    crstldisc_topo = keras.Model(input_img, output)
    crstldisc_topo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #crstldisc.summary()
    return crstldisc_topo

In [20]:
def define_generator():
    #Generator
    visible = keras.layers.Input(shape=(128,))
    n_nodes = 128 * 16 * 10
    dense = keras.layers.Dense(n_nodes)(visible)
    reshape1 = keras.layers.Reshape((10, 16, 128))(dense)
    upsample1 = keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(reshape1)
    LeakyRelu1 = keras.layers.LeakyReLU(alpha=0.2)(upsample1)
    upsample2 = keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(LeakyRelu1)
    LeakyRelu2 = keras.layers.LeakyReLU(alpha=0.2)(upsample2)
    upsample3 = keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(LeakyRelu2)
    LeakyRelu3 = keras.layers.LeakyReLU(alpha=0.2)(upsample3)
    upsample4 = keras.layers.Conv2DTranspose(64, (2,2), strides=(1,4), padding='same')(LeakyRelu3)
    LeakyRelu4 = keras.layers.LeakyReLU(alpha=0.2)(upsample4)
    conv_layer1 = Conv2D(1, (2,2), activation='sigmoid', padding='same')(LeakyRelu4)
    #output = crstldisc(conv_layer1)
    generator = keras.Model(visible, conv_layer1)
    generator.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
    #generator.summary()
    return generator

In [21]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(128,n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = np.zeros((n_samples, 1))
    return X[:,:,:,0], y

In [23]:
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    input_img = keras.Input(shape=( 128,))
    # add generator
    gen = g_model(input_img)
    # add the discriminator
    disc = d_model(gen)
    # compile model
    opt = Adam(learning_rate=0.0005, beta_1=0.5)
    model = keras.Model(inputs=input_img, outputs=[disc])
    model.compile(loss=['binary_crossentropy'], optimizer=opt)
    return model
def define_gan_topo(g_model, dtopo_model):
    # make weights in the discriminator not trainable
    dtopo_model.trainable = False
    # connect them
    input_img = keras.Input(shape=( 128,))
    # add generator
    gen = g_model(input_img)
    # add the discriminator
    topo = dtopo_model(gen)
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model = keras.Model(inputs=input_img, outputs=[topo])
    model.compile(loss=['binary_crossentropy'], optimizer=opt)
    return model

In [25]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, n_samples, x_input):
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = np.zeros((n_samples, 1))
    return X[:,:,:,0], y

In [26]:
def train(g_model, d_model, dtopo_model, dataset, xtopodataset, ytopodataset, n_epochs=200, n_batch=256):
    half_batch = int(n_batch / 2)
    g1_loss=[]
    g2_loss=[]
    # manually enumerate epochs
    for i in range(10):
    # enumerate batches over the training set
        for j in range(bat_per_epo):
            X_gan=generate_latent_points(128,half_batch)
            ##########################################################
            # get randomly selected 'real' samples
            if i<99:
                X_real, y_real = generate_real_samples(dataset, half_batch)
                # generate 'fake' examples
                #g_model=keras.models.load_model('g_model4.keras')
                X_fake, y_fake = generate_fake_samples(g_model, half_batch, X_gan)
                # create training set for the discriminator
                X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
                # update discriminator model weights
                if i>0 or j>0:
                    d_model=keras.models.load_model('d_model4.keras')
                d_loss, d_acc = d_model.train_on_batch(X, y)
                d_model.save('d_model.keras')
            ##########################################################
            # gen real topo samples
            if i>99:
                X_real, y_real = generate_topo_samples(xtopodataset, ytopodataset, half_batch)
                # create training set for the discriminator
                X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
                #update discriminator model weights
                if i>0 or j>0:
                    dtopo_model=keras.models.load_model('dtopo_model.keras')
                d_losstopo, d_acctopo = dtopo_model.train_on_batch(X, y)
                dtopo_model.save('dtopo_model4keras')
            ##########################################################
            # create inverted labels for the fake samples
            if j>0:
                g_model=keras.models.load_model('g_model.keras')
            #Train Generator, phase I
            if i<100:
                gan_model = define_gan(g_model, d_model)
            if i>=100:
                gan_model = define_gan_topo(g_model, dtopo_model)
            #Train Topology
            y_gan = np.ones((half_batch, 1))
            # update the generator via the discriminator's error
            g_loss1 = gan_model1.train_on_batch(X_gan, [y_gan])
            print("GAN_1 Loss: "+str(g_loss1))
            g1_loss.append(g_loss1)
            np.save('g1_loss.npy',np.array(g1_loss))
            g_model.save('g_model.keras')
        #Archive models evry 10 epochs
        if i%10==0
            g_model.save('g_model_Epoch_'+str(i)+'.keras')
       
    
        

In [ ]:
#Learning rat changed to .0001
g_model = define_generator()
#g_model=keras.models.load_model('g_model_Epoch_9.keras')
g_model.save('g_model.keras')
d_model = define_discriminator()
#d_model=keras.models.load_model('d_model_Epoch_9.keras')
dtopo_model = define_topo_discriminator()
#dtopo_model=keras.models.load_model('dtopo_model_Epoch_109.keras')
#dtopo_model.save('dtopo_model.keras')
traint(g_model, d_model, dtopo_model, x_real_train, x_train_topo, y_train_topo, n_epochs=200, n_batch=256)
